### Disclaimer

This software is provided solely for educational and research purposes. 
It is not intended to provide investment advice, and no investment recommendations are made herein. 
The developers are not financial advisors and accept no responsibility for any financial decisions or losses resulting from the use of this software. 
Always consult a professional financial advisor before making any investment decisions.

### Imports

In [19]:
import yfinance as yf
from datetime import datetime, timedelta
from scipy.interpolate import interp1d
import numpy as np
import argparse
import warnings
import pandas as pd
import os
import requests
import plotly.express as px
from pandas.tseries.offsets import DateOffset
import plotly.io as pio
pio.renderers.default = "notebook_connected"

pd.set_option('display.float_format', '{:.6f}'.format)
pd.options.display.max_columns = 30
warnings.filterwarnings("ignore", message="Not enough unique days to interpolate for ticker")

In [10]:
ticker = "META"
ticker_obj = yf.Ticker(ticker)

In [12]:
%matplotlib inline

df_flat["text"] = (df_flat["open_pct_change"] * 100).round(3).astype(str) + "%"

p = px.bar(
    x=df_flat["Date"],
    y=df_flat["open_pct_change"].round(3),
    color=df_flat.index.astype(str),
    title="Open % Change",
    text=df_flat["text"]
)

p.update_traces(textangle=0)

p.show()

In [278]:
result = compute_recommendation(ticker)

/var/folders/mn/xphw12rn1_zfln59d9xj_qvw0000gn/T/ipykernel_15430/3843329786.py:319: UserWarning:

For ticker META straddle is either 0 or None from available bid/ask spread... using lastPrice.



64
24.31


In [324]:
def calc_prev_avg_earnings_move(df_history, ticker_obj, days_back=EARNINGS_LOOKBACK_DAYS_FOR_AGG, plot_loc=PLOT_LOC):
    df_history = df_history.copy()
    if "Date" not in df_history.columns and df_history.index.name == "Date":
        df_history = df_history.reset_index()
    df_history["Date"] = df_history["Date"].dt.date
    df_history = df_history.sort_values("Date")

    df_earnings_dates = ticker_obj.earnings_dates.reset_index()
    df_earnings_dates = df_earnings_dates[df_earnings_dates["Event Type"] == "Earnings"].copy()
    df_earnings_dates["Date"] = df_earnings_dates["Earnings Date"].dt.date

    def classify_release(dt):
        hour = dt.hour
        if hour < 9:
            return "pre-market"
        elif hour >= 9:
            return "post-market"

    df_earnings_dates["release_timing"] = df_earnings_dates["Earnings Date"].apply(classify_release)
    df_earnings = df_earnings_dates.merge(df_history, on="Date", how="left", suffixes=('', '_earnings'))
    df_earnings["next_date"] = df_earnings["Date"] + pd.Timedelta(days=1)
    df_next = df_history.rename(columns=lambda c: f"{c}_next" if c != "Date" else "next_date")
    df_flat = df_earnings.merge(df_next, on="next_date", how="left")
    df_flat["prev_close"] = df_flat["Close"].shift(1)
    df_flat["pre_market_move"] = (df_flat["Open"] - df_flat["prev_close"]) / df_flat["prev_close"]
    df_flat["post_market_move"] = (df_flat["Open_next"] - df_flat["Close"]) / df_flat["Close"]
    
    df_flat["earnings_move"] = df_flat.apply(
        lambda row: row["pre_market_move"] if row["release_timing"] == "pre-market"
        else row["post_market_move"] if row["release_timing"] == "post-market"
        else None,axis=1
    )

    if plot_loc:
        df_flat["text"] = (df_flat["earnings_move"] * 100).round(2).astype(str) + "%"
        p = px.bar(
            x=df_flat["Date"],
            y=df_flat["earnings_move"].round(3),
            color=df_flat.index.astype(str),
            text=df_flat["text"],
            title="Earnings % Move",
        )
        p.update_traces(textangle=0)
        # p.show()

        full_path = os.path.join(plot_loc, f"{ticker}_{df_flat["Date"].iloc[0].strftime("%Y-%m-%d")}.html")
        os.makedirs(plot_loc, exist_ok=True)
        p.write_html(full_path)
        print(f"Saved plot for ticker {ticker} here: {full_path}")

    avg_abs_pct_move = round(abs(df_flat["earnings_move"]).mean(), 3)
    median_abs_pct_move = round(abs(df_flat["earnings_move"]).median(), 3)
    min_abs_pct_move = round(abs(df_flat["earnings_move"]).min(), 3)
    max_abs_pct_move = round(abs(df_flat["earnings_move"]).max(), 3)
    return avg_abs_pct_move, median_abs_pct_move, min_abs_pct_move, max_abs_pct_move, df_flat

In [331]:
a,b,c,d,df_flat = calc_prev_avg_earnings_move(df_history, ticker_obj)

In [332]:
df_flat

,Earnings Date,EPS Estimate,Reported EPS,Surprise(%),Event Type,Date,release_timing,Open,High,Low,Close,Volume,Dividends,Stock Splits,next_date,Open_next,High_next,Low_next,Close_next,Volume_next,Dividends_next,Stock Splits_next,prev_close,pre_market_move,post_market_move,earnings_move


In [333]:
df_flat["text"] = (df_flat["earnings_move"] * 100).round(2).astype(str) + "%"

In [336]:
px.bar(
    data_frame=df_flat,
    x="Date",
    y="earnings_move",
    color=df_flat.index.astype(str),
    text=df_flat["text"],
    title="Earnings % Move"
)